The purpose of this notebook is to evaluate the model to be used by the QA Bot accelerator.  This notebook is available at https://github.com/databricks-industry-solutions/diy-llm-qa-bot.

## Evaluating QA model performance
In this notebook, we showcase how to evaluate QA performance using langchain's `QAEvalChain` using a evaluation set containing correct references and responses. We use an LLM as a grader to compare model responses with the correct responses.

In [0]:
%run "./utils/config_utils"

## Scoring the evaluation set

We first retrieve and install the Python dependencies from the logged model in order to score the model. 

mlflow writes a file with model dependencies at `requirements_path` in DBFS. We then use %pip to install the dependencies in the file.

In [0]:
import mlflow

requirements_path = mlflow.pyfunc.get_model_dependencies(config['model_uri'])
%pip install -r $requirements_path
dbutils.library.restartPython()

2023/08/11 01:29:46 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmp4h_4m90f/requirements.txt'.


Python interpreter will be restarted.
  Using cached faiss_cpu-1.7.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached langchain-0.0.250-py3-none-any.whl (1.4 MB)
  Using cached openai-0.27.6-py3-none-any.whl (71 kB)
  Using cached tiktoken-0.4.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached typing_inspect-0.8.0-py3-none-any.whl (8.7 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached langsmith-0.0.21-py3-none-any.whl (32 kB)
  Using cached numexpr-2.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (382 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached SQLAlchemy-2.0.19-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl (26 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.

Because the previous block restarts the kernel, let's set our configs again and import the dependencies:

In [0]:
%run "./utils/config_utils"

In [0]:
import pandas as pd
import numpy as np
import openai
import json
from langchain.llms import OpenAI
import os
from langchain.evaluation.qa import QAEvalChain
import mlflow

We have prepared an evaluation set of questions and correct answers that contain both the correct reference and a sample answer. Let's take a look at an example:

In [0]:
eval_dataset = (
  spark
    .table('sources')
    .selectExpr(
        "title",
        "url",
        "answer"
    )
    .withColumnRenamed("title", "question")
    .withColumnRenamed("url", "correct_source")
    .sample(0.05, seed=42).toPandas())

eval_dataset

,question,correct_source,answer
0,¿El cónsul puede obligar a comparecer al reque...,https://www.icbf.gov.co/el-consul-puede-obliga...,"No, el cónsul no cuenta con la facultad de obl..."
1,¿Cómo logra el ICBF que sus programas sean inc...,https://www.icbf.gov.co/como-logra-el-icbf-que...,A través de la creación de un modelo de enfoqu...
2,¿Cuáles son los objetivos del enfoque diferenc...,https://www.icbf.gov.co/cuales-son-los-objetiv...,El objetivo principal del enfoque diferencial ...
3,¿Qué es la homofobia?,https://www.icbf.gov.co/que-es-la-homofobia,La homofobia es el rechazo constante de person...
4,¿Cuáles son las acciones que se realizan para ...,https://www.icbf.gov.co/cuales-son-las-accione...,"1. Conocer y entender las características, con..."
5,¿A quién le corresponde el pago de la prueba d...,https://www.icbf.gov.co/quien-le-corresponde-e...,El costo será asumido por quien solicita la pr...
6,Quiénes pueden ser contratados por el ICBF par...,https://www.icbf.gov.co/quienes-pueden-ser-con...,"""""""Dado el interés del ICBF por garantizar que..."
7,¿Cuál es el procedimiento para iniciar un proc...,https://www.icbf.gov.co/cual-es-el-procedimien...,El Defensor de Familia cita a las personas inv...
8,¿Cuáles son las implicaciones legales de reali...,https://www.icbf.gov.co/cuales-son-las-implica...,Se establece la filiación legal entre el padre...
9,"Realizamos conciliación frente a alimentos, cu...",https://www.icbf.gov.co/realizamos-conciliacio...,Cuando se incumplan los acuerdos fijados media...


In [0]:
# Rephrasing the questions to test the AI model
rephrasing = ['¿Puede un cónsul obligar a comparecer a alguien en un proceso?',
              '¿El ICBF es incluyente con las personas con discapacidad?',
              '¿Qué es el enfoque diferencial?',
              'Definición de homofobia',
              '¿Cómo se logra el enfoque diferencial?',
              '¿Quién paga las pruebas de ADN?',
              '¿A quiénes contrata el ICBF para atender personas indígenas o afro',
              '¿Cómo hago para reconocer la paternidad de mi hija?',
              '¿Qué sucede cuando reconozco la paternidad de mi hije?',
              '¿Cómo puedo ver a mi hije si la persona con la custodia no me deja verle?',
              '¿Cómo puedo cumplir con la oferta de alimentos si no vivo con mi hije?',
              '¿Los datos personales que recolecta el ICBF son públicos?',
              '¿Un servidor público debe recibir dinero para adelantar trámites?',
              '¿Me puedo negar a hacerme la prueba de ADN?',
              '¿Puedo presentar una prueba de ADN que me haya tomado yo?', 
              'Quiero adoptar a un adulto, ¿cómo lo puedo hacer?',
              'Soy extranjera en Colombia y quiero adoptar, ¿adónde acudo?',
              '¿Cómo puedo adoptar un niño extranjero?']

eval_dataset['validation_question'] = rephrasing
eval_dataset = eval_dataset.reset_index(drop=True).to_dict('records')
eval_dataset

Out[70]: [{'question': '¿El cónsul puede obligar a comparecer al requerido en un proceso?',
  'correct_source': 'https://www.icbf.gov.co/el-consul-puede-obligar-comparecer-al-requerido-en-un-proceso',
  'answer': 'No, el cónsul no cuenta con la facultad de obligar a ninguna persona para la ejecución de diligencias para las cuales ha sido comisionado. Su actuación deberá enmarcarse dentro de la más absoluta moderación y prudencia, evitando así cualquier inconveniente que pueda afectar las relaciones internacionales entre los estados. ',
  'validation_question': '¿Puede un cónsul obligar a comparecer a alguien en un proceso?'},
 {'question': '¿Cómo logra el ICBF que sus programas sean inclusivos frente a la atención de niños, niñas y adolescentes con discapacidad y sus familias?',
  'correct_source': 'https://www.icbf.gov.co/como-logra-el-icbf-que-sus-programas-sean-inclusivos-frente-la-atencion-de-ninos-ninas-y',
  'answer': 'A través de la creación de un modelo de enfoque diferencial d

In [0]:
queries = pd.DataFrame({'question': rephrasing})
model = mlflow.pyfunc.load_model(config['model_uri'])
predictions = model.predict(queries)
predictions[0]

2023/08/11 02:37:53 WARNING mlflow.openai.retry: Retrying openai.api_resources.embedding.Embedding.create in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')).


Out[65]: {'answer': 'No, según el contexto, el cónsul no cuenta con la facultad de obligar a ninguna persona a comparecer en un proceso. Su actuación debe ser moderada y prudente, evitando cualquier inconveniente que pueda afectar las relaciones internacionales entre los estados.',
 'source': 'https://www.icbf.gov.co/el-consul-puede-obligar-comparecer-al-requerido-en-un-proceso',
 'output_metadata': {'token_usage': {'prompt_tokens': 265,
   'completion_tokens': 60,
   'total_tokens': 325},
  'model_name': 'gpt-3.5-turbo'}}

langchain's `QAEvalChain` acts as a grader: compares whether the scored answers are sufficiently similar to the ground truth and returns CORRECT or INCORRECT for each question.

In [0]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(eval_dataset, predictions, question_key="validation_question", prediction_key="answer")
graded_outputs

Out[71]: [{'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' INCORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'}]

We can compile the graded results, the scored answers and the eval dataset back into one DataFrame. Note that the bot can sometimes produce correct answers based on different reference sources.

In [0]:
results = pd.DataFrame(
  [{
    "question": eval_data["question"], 
    "prediction": predict["answer"], 
    "source": predict["source"], 
    "correct_source": eval_data["correct_source"], 
    "answer": eval_data["answer"], 
    "find_correct_source": predict["source"] == eval_data["correct_source"], 
    "same_as_answer": True if graded_outputs["results"].strip() == 'CORRECT' else False
    } 
    for (predict, eval_data, graded_outputs) in zip(predictions, eval_dataset, graded_outputs)])
display(spark.createDataFrame(results))

/databricks/spark/python/pyspark/sql/pandas/conversion.py:656: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


question,prediction,source,correct_source,answer,find_correct_source,same_as_answer
¿El cónsul puede obligar a comparecer al requerido en un proceso?,"No, según el contexto, el cónsul no cuenta con la facultad de obligar a ninguna persona a comparecer en un proceso. Su actuación debe ser moderada y prudente, evitando cualquier inconveniente que pueda afectar las relaciones internacionales entre los estados.",https://www.icbf.gov.co/el-consul-puede-obligar-comparecer-al-requerido-en-un-proceso,https://www.icbf.gov.co/el-consul-puede-obligar-comparecer-al-requerido-en-un-proceso,"No, el cónsul no cuenta con la facultad de obligar a ninguna persona para la ejecución de diligencias para las cuales ha sido comisionado. Su actuación deberá enmarcarse dentro de la más absoluta moderación y prudencia, evitando así cualquier inconveniente que pueda afectar las relaciones internacionales entre los estados.",true,true
"¿Cómo logra el ICBF que sus programas sean inclusivos frente a la atención de niños, niñas y adolescentes con discapacidad y sus familias?","Sí, el ICBF es incluyente con las personas con discapacidad, ya que en el texto se menciona que el enfoque diferencial del ICBF se aplica a niños, niñas y adolescentes con discapacidad y derechos vulnerados.",https://www.icbf.gov.co/quienes-aplica-el-icbf-el-enfoque-diferencial-1,https://www.icbf.gov.co/como-logra-el-icbf-que-sus-programas-sean-inclusivos-frente-la-atencion-de-ninos-ninas-y,"A través de la creación de un modelo de enfoque diferencial de derechos que promueve en sus programas la inclusión de niños, niñas, adolescentes con discapacidad y sus familias, con lo cual se garantiza la participación activa de esta población.",false,true
¿Cuáles son los objetivos del enfoque diferencial?,"El enfoque diferencial es una estrategia que busca promover la igualdad social en la sociedad colombiana, reconociendo, comprendiendo y valorando las diferencias culturales y sociales de los ciudadanos. Sus objetivos incluyen favorecer el bienestar y desarrollo de comunidades culturalmente diversas, proteger los derechos de grupos considerados minorías, reducir la discriminación y diseñar políticas que brinden igualdad de oportunidades a grupos históricamente marginados.",https://www.icbf.gov.co/cuales-son-los-objetivos-del-enfoque-diferencial-1,https://www.icbf.gov.co/cuales-son-los-objetivos-del-enfoque-diferencial,"El objetivo principal del enfoque diferencial es que la sociedad colombiana avance hacia la igualdad social, a partir de reconocer, comprender y valorar las creencias, prácticas, saberes, expectativas y formas propias de relacionamiento de los ciudadanos. Sus otros objetivos son: Favorecer y hacer posible el bienestar y desarrollo de los pueblos, comunidades y colectivos culturalmente diversos. Proteger los derechos de los grupos e individuos considerados como minoría. Contribuir a la reducción de cualquier forma de discriminación o segregación social, cultural, espiritual, económica y espacial. Diseñar e implementar políticas, programas o servicios que ofrezcan igualdad de oportunidades de desarrollo a grupos tradicionalmente segregados, excluidos o marginados social y culturalmente.",false,true
¿Qué es la homofobia?,"La homofobia se define como el rechazo constante de personas heterosexuales hacia personas que son homosexuales o bisexuales. Puede manifestarse de diferentes maneras, como la invisibilización de las personas LGTBI o los crímenes de odio hacia ellos. La homofobia se basa en la creencia errónea de que una persona heterosexual es supuestamente superior a las personas homosexuales o bisexuales.",https://www.icbf.gov.co/que-es-la-homofobia,https://www.icbf.gov.co/que-es-la-homofobia,"La homofobia es el rechazo constante de personas heterosexuales a personas que son homosexuales o bisexuales. Puede manifestarse de varias formas, como, por ejemplo, la invisibilización de las personas LGTBI o los crímenes de odio hacia ellos. La homofobia se basa en el pensamiento errado 

In [0]:
results['same_as_answer'].sum() / len(results)

Out[73]: 0.9444444444444444

Our QA bot seems to produce sensible responses most of the time according to the LLM grader. However, it is still important for the developer to regularly evaluate the performance by reading the responses. It is often possible for the LLM to miss nuanced differences between concepts and produce false negative gradings, especially when answers are long and complex.  And make sure to review the eval question set periodically so that it reflects the type of questions the users submit.

© 2023 Databricks, Inc. All rights reserved. The source in this notebook is provided subject to the Databricks License. All included or referenced third party libraries are subject to the licenses set forth below.

| library                                | description             | license    | source                                              |
|----------------------------------------|-------------------------|------------|-----------------------------------------------------|
| langchain | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/langchain/ |
| tiktoken | Fast BPE tokeniser for use with OpenAI's models | MIT  |   https://pypi.org/project/tiktoken/ |
| faiss-cpu | Library for efficient similarity search and clustering of dense vectors | MIT  |   https://pypi.org/project/faiss-cpu/ |
| openai | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/openai/ |